# Fact Manutenção

Tabela fato de manutenção com cálculo de downtime.

In [ ]:
# Configuração (lê parâmetros do job)
CATALOG = dbutils.widgets.get('catalog') if 'catalog' in [w.key for w in dbutils.widgets.getAll()] else 'manufatura_lakehouse'
SCHEMA_SILVER = dbutils.widgets.get('schema_silver') if 'schema_silver' in [w.key for w in dbutils.widgets.getAll()] else 'silver'
SCHEMA_GOLD = dbutils.widgets.get('schema_gold') if 'schema_gold' in [w.key for w in dbutils.widgets.getAll()] else 'gold'

def fqtn(schema, table):
    if CATALOG and CATALOG.lower() != 'none':
        return f"`{CATALOG}`.`{schema}`.`{table}`"
    else:
        return f"`{schema}`.`{table}`"

# Criar schema se necessário
if CATALOG and CATALOG.lower() != 'none':
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{CATALOG}`.`{SCHEMA_GOLD}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA_GOLD}`")


In [ ]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {fqtn(SCHEMA_GOLD, 'fact_manutencao')} (
    maintenance_sk STRING,
    equipment_id STRING,
    start_date_key INT,
    maintenance_type STRING,
    actual_start TIMESTAMP,
    actual_end TIMESTAMP,
    downtime_minutes DOUBLE,
    technician_id STRING,
    cost_estimate DOUBLE
) USING DELTA
PARTITIONED BY (start_date_key)
""")

spark.sql(f"""
INSERT OVERWRITE {fqtn(SCHEMA_GOLD, 'fact_manutencao')}
SELECT 
    maintenance_order_id as maintenance_sk,
    equipment_id,
    cast(date_format(actual_start, 'yyyyMMdd') as int) as start_date_key,
    maintenance_type,
    actual_start,
    actual_end,
    (unix_timestamp(actual_end) - unix_timestamp(actual_start))/60 as downtime_minutes,
    technician_id,
    ((unix_timestamp(actual_end) - unix_timestamp(actual_start))/3600) * 100 as cost_estimate
FROM {fqtn(SCHEMA_SILVER, 'maintenance_orders_clean')}
WHERE actual_start IS NOT NULL
""")